In [2]:
# pip install openai
!pip3 install openai

#!pip3 install --upgrade pip

In [3]:
from google.colab import drive
import os

drive.mount('/content/drive')
base_dir = '/content/drive/MyDrive/Homework3'
prompt_dir = os.path.join(base_dir, 'prompts')
os.makedirs(prompt_dir, exist_ok=True)

Mounted at /content/drive


In [4]:
import os
from openai import OpenAI
import pandas as pd
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
# Set it as an environment variable
os.environ["GITHUB_TOKEN"] = "omitted" # my personal key
# To authenticate with the model you will need to generate a personal access token (PAT) in your GitHub settings.
client = OpenAI(
    base_url="https://models.inference.ai.azure.com",
    api_key=os.environ["GITHUB_TOKEN"],
)

# List of available models
model_choices = ["Codestral-2501, gpt-4o-mini"]
#DeepSeek is not allowed on gov owened machines

    1: "Zero-Shot",
    2: "Few-Shot",
    3: "Chain-of-Thought",
    4: "Self-Consistency",
    5: "Prompt Chaining",
    6: "Role-Playing"

In [ ]:
import os

# Start checkpoint
start_task = 21
prompt_dir = "/content/drive/MyDrive/Homework3/prompts"

# Format: (task, strategy, first or second, prompt type)
prompts = [
  # Task 21
  (21, 1, 3, "Read through this script and figure out what could go wrong logically. Then rewrite it to be more correct and easier to follow."),
  (21, 2, 6, "Pretend as if you're doing a code review on me. Identify any potential bugs or design flaws in this Python utility, then suggest a cleaner version that fixes those issues."),

  # Task 22
  (22, 1, 5, "First figure out what this script is trying to do and then improve it so it cleans each line properly and counts words without error."),
  (22, 2, 2, "Example: Input-> 'Hello, world!' -> ['hello', 'world'] Now finish this file-processing function to clean text and count words reliably."),
]

# Process prompts
for task, strategy, prompt_type, prompt_text in prompts:
    base_filename = f"task{task:02d}_strategy{strategy}"
    new_filename = f"{base_filename}-{prompt_type}.txt"
    new_filepath = os.path.join(prompt_dir, new_filename)

    # Write prompt
    with open(new_filepath, "w") as f:
        f.write(prompt_text.strip())
    print(f"Saved: {new_filename}")

    # Remove any old file
    old_filepath = os.path.join(prompt_dir, f"{base_filename}.txt")
    if os.path.exists(old_filepath):
        os.remove(old_filepath)
        print(f"Removed old file: {base_filename}.txt")


Saved: task21_strategy1-3.txt
Removed old file: task21_strategy1.txt
Saved: task21_strategy2-6.txt
Removed old file: task21_strategy2.txt
Saved: task22_strategy1-5.txt
Removed old file: task22_strategy1.txt
Saved: task22_strategy2-2.txt
Removed old file: task22_strategy2.txt


In [ ]:
# Paths defined
code_csv_path = "/content/drive/MyDrive/Homework3/assignment3_task_code_formatted.csv"
prompts_path = "/content/drive/MyDrive/Homework3/prompts"

# Read the CSV of code samples
code_df = pd.read_csv(code_csv_path)

# List all prompt files
prompt_files = sorted(f for f in os.listdir(prompts_path) if f.endswith(".txt"))

# Strategy label mapping
strategy_map = {
    1: "Zero-Shot",
    2: "Few-Shot",
    3: "Chain-of-Thought",
    4: "Self-Consistency",
    5: "Prompt Chaining",
    6: "Role-Playing"
}

# model_choices[0] = "Codestral, gpt-4o-mini"
codestral_model = model_choices[0].split(",")[0].strip()
gpt4o_model = model_choices[0].split(",")[1].strip()

records = []

# Loop through all prompts
for fname in prompt_files:
    parts = fname.replace(".txt", "").split("_")
    task_num = int(parts[0].replace("task", ""))
    strat_num = int(parts[1].split("-")[1])
    strategy_label = strategy_map.get(strat_num, f"Strategy-{strat_num}")

    # Read prompt text
    with open(os.path.join(prompts_path, fname), "r") as f:
        prompt_text = f.read().strip()

    # Lookup corresponding code snippet
    code_block = code_df.iloc[task_num - 1]["Code"]
    full_input = f"{prompt_text}\n\n```python\n{code_block}\n```"

    # Model 1: Codestral-2501
    try:
        codestral_response = client.chat.completions.create(
            model=codestral_model,
            messages=[{"role": "user", "content": full_input}],
            max_tokens=1024,
            temperature=0.7,
        ).choices[0].message.content.strip()
    except Exception as e:
        codestral_response = f"[ERROR from {codestral_model}]: {e}"

    # Model 2: GPT-4o-mini
    try:
        gpt4o_response = client.chat.completions.create(
            model=gpt4o_model,
            messages=[{"role": "user", "content": full_input}],
            max_tokens=1024,
            temperature=0.7,
        ).choices[0].message.content.strip()
    except Exception as e:
        gpt4o_response = f"[ERROR from {gpt4o_model}]: {e}"

    # Save both model outputs side-by-side
    records.append({
        "Task": f"{task_num:02d}",
        "Strategy": strategy_label,
        "Temperature": 0.7,
        "Prompt": prompt_text,
        "Codestral Output": codestral_response,
        "GPT-4o-mini Output": gpt4o_response,
    })

# Save to CSV
output_path = "/content/drive/MyDrive/Homework3/Assignment3_Results.csv"
df = pd.DataFrame(records)
df.to_csv(output_path, index=False)

print(f"Results in: {output_path}")


Results in: /content/drive/MyDrive/Homework3/Assignment3_Results.csv


# Update Table to Calculate BLEU scores between models

In [5]:
# config
input_csv = "/content/drive/MyDrive/Homework3/Assignment3_Results.csv"
output_csv = "/content/drive/MyDrive/Homework3/Assignment3_Results_WithBLEU.csv"

# load data
df = pd.read_csv(input_csv)

# function to compute bleu
def compute_bleu(reference, candidate):
    reference_tokens = reference.split()
    candidate_tokens = candidate.split()
    smoothie = SmoothingFunction().method4
    try:
        score = sentence_bleu([reference_tokens], candidate_tokens, smoothing_function=smoothie)
    except ZeroDivisionError:
        score = 0.0
    return score

# Compute bleu for each row
bleu_scores = []
for idx, row in df.iterrows():
    ref_output = str(row['GPT-4o-mini Output'])
    cand_output = str(row['Codestral-2501 Output'])
    bleu = compute_bleu(ref_output, cand_output)
    bleu_scores.append(bleu)

# Add new column
df['BLEU Score'] = bleu_scores

# save updated csv
df.to_csv(output_csv, index=False)
print(f"Saved updated file with BLEU scores: {output_csv}")

Saved updated file with BLEU scores: /content/drive/MyDrive/Homework3/Assignment3_Results_WithBLEU.csv
